In [86]:
import pandas as pd

import numpy as np


In [87]:
predDf = pd.read_csv(
    "/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_2/case_study_pred.csv"
)
gtDf = pd.read_csv(
    "/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_2/case_study_gt.csv"
)
visionPredDf1 = pd.read_csv("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/saloon_focal_480_aug_2/Saloon - 4 Dr_imgs_pred_output.csv")
visionPredDf2 = pd.read_csv("//home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/hatchback_focal_480_aug/Hatchback - 5 Dr_imgs_pred_output.csv")
visionPredDf3 = pd.read_csv("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/suv_focal_480_aug/SUV - 5 Dr_imgs_pred_output.csv")
visionPredDf = pd.concat([visionPredDf1, visionPredDf2,visionPredDf3 ])
majorParts = [x.replace("vision_", "") for x in  visionPredDf.columns]
majorParts

['Unnamed: 0',
 'CaseID',
 'bonnet',
 'bumper_front',
 'door_front_rh',
 'door_rear_rh',
 'engine',
 'fender_front_rh',
 'front_panel',
 'headlamp_rh',
 'non_external',
 'misc',
 'wheel',
 'windscreen_front',
 'bumper_rear',
 'rear_compartment',
 'rear_panel',
 'rear_quarter_rh',
 'tail_lamp_rh',
 'door_front_lh',
 'door_rear_lh',
 'rear_quarter_lh',
 'tail_lamp_lh',
 'fender_front_lh',
 'headlamp_lh',
 'grille']

In [88]:
len(predDf)

30

In [89]:
targetCols = [x for x in predDf.columns if "0" not in x and x != "CaseID"]
allRows = []
for c in targetCols:
    for caseId in predDf["CaseID"].unique().tolist():
        visionPred = -1
        if(c in majorParts):
            visionPred = visionPredDf[visionPredDf["CaseID"] == caseId][f"vision_{c}"].item()
            

        pred = predDf[predDf["CaseID"] == caseId][c].item()
        gt = gtDf[gtDf["CaseID"] == caseId][c].item()
        fp = 1 if gt == 0 and pred == 1 else 0
        fn = 1 if gt == 1 and pred == 0 else 0
        allRows.append(
            {
                "CaseID" : caseId,
                "part" : c,
                "v_pred":visionPred,
                "pred" : pred,
                "gt" : gt,
                "fp" : fp, 
                "fn" : fn,
                "inspect" : None,
                "remarks" : None

            }
        )
caseStudyInspectDf = pd.json_normalize(allRows)
caseStudyInspectDf.sort_values(by = ["CaseID", "part"], inplace=True)

In [90]:
caseStudyInspectDf

,CaseID,part,v_pred,pred,gt,fp,fn,inspect,remarks
259,10053736,air_conditioning,-1,1,0,1,0,None,None
439,10053736,airbag,-1,1,0,1,0,None,None
19,10053736,bonnet,1,1,1,0,0,None,None
49,10053736,bumper_front,1,1,1,0,0,None,None
349,10053736,bumper_rear,1,1,0,1,0,None,None
...,...,...,...,...,...,...,...,...,...
752,13127647,tail_lamp_rh,0,0,0,0,0,None,None
482,13127647,undercarriage,-1,0,0,0,0,None,None
512,13127647,wheel,0,0,0,0,0,None,None
782,13127647,windscreen_front,0,0,0,0,0,None,None


In [91]:
len(caseStudyInspectDf["part"].unique())

34

In [92]:
majorParts = 25 * 30
majorParts

750

In [93]:
154 / majorParts

0.20533333333333334

In [94]:
caseStudyWrong = caseStudyInspectDf[(caseStudyInspectDf["pred"] !=caseStudyInspectDf["gt"] )
                                    & (caseStudyInspectDf["v_pred"] != -1)
                                    
                                    ]
caseStudyWrong

,CaseID,part,v_pred,pred,gt,fp,fn,inspect,remarks
349,10053736,bumper_rear,1,1,0,1,0,None,None
409,10053736,engine,1,1,0,1,0,None,None
559,10053736,fender_front_lh,1,1,0,1,0,None,None
139,10053736,headlamp_lh,1,1,0,1,0,None,None
379,10053736,misc,1,1,0,1,0,None,None
...,...,...,...,...,...,...,...,...,...
381,13045105,misc,1,1,0,1,0,None,None
32,13127647,bumper_front,1,1,0,1,0,None,None
332,13127647,bumper_rear,1,1,0,1,0,None,None
182,13127647,door_front_lh,1,1,0,1,0,None,None


In [95]:
len(caseStudyWrong) / len(caseStudyInspectDf)

0.11470588235294117

In [96]:

caseStudyWrongVis = caseStudyInspectDf[(caseStudyInspectDf["pred"] !=caseStudyInspectDf["v_pred"]) & (caseStudyInspectDf["v_pred"] != -1) ]
caseStudyWrongVis

,CaseID,part,v_pred,pred,gt,fp,fn,inspect,remarks
159,10231429,headlamp_rh,1,0,0,0,0,None,None


In [97]:
# majorPartDf = caseStudyInspectDf[caseStudyInspectDf["v_pred"] != - 1]
majorPartDf = caseStudyInspectDf

In [98]:
majorPartDf["part"].unique()

array(['air_conditioning', 'airbag', 'bonnet', 'bumper_front',
       'bumper_rear', 'cooling_fan', 'door_front_lh', 'door_front_rh',
       'door_mirror_lh', 'door_mirror_rh', 'door_rear_lh', 'door_rear_rh',
       'engine', 'fender_front_lh', 'fender_front_rh', 'fog_lamp_lh',
       'fog_lamp_rh', 'front_panel', 'grille', 'headlamp_lh',
       'headlamp_rh', 'interior', 'misc', 'radiator', 'rear_compartment',
       'rear_panel', 'rear_quarter_lh', 'rear_quarter_rh', 'tail_lamp_lh',
       'tail_lamp_rh', 'undercarriage', 'wheel', 'windscreen_front',
       'windscreen_rear'], dtype=object)

In [99]:
fpr = len(majorPartDf[(majorPartDf["pred"]  == 1) & (majorPartDf["gt"]  == 0) ]) / len(majorPartDf[ (majorPartDf["gt"]  == 0)])
fpr

0.17490952955367914

In [100]:
fnr = len(majorPartDf[(majorPartDf["pred"]  == 0) & (majorPartDf["gt"]  == 1) ]) / len(majorPartDf[ (majorPartDf["gt"]  == 1)])
fnr

0.2356020942408377

In [101]:
acc = len(majorPartDf[(majorPartDf["pred"]  == majorPartDf["gt"])]) / len(majorPartDf)
acc

0.8137254901960784

In [102]:
allSubsetAcc = []
for caseId in majorPartDf["CaseID"].unique().tolist():
    rows = majorPartDf[majorPartDf["CaseID"] == caseId]
    correct = rows[rows["pred"] == rows["gt"]]
    subset_acc = len(correct) / len(rows)
    allSubsetAcc.append(subset_acc)
    
    # print(subset_acc)
print(np.mean(allSubsetAcc))

0.8137254901960784


In [103]:
caseStudyInspectDf.to_csv("./case_study_result.csv")